In [53]:
from pyvis.network import Network
from pyvis import network as net
import pandas as pd
from io import StringIO

In [73]:
csvString = """
Source,Target,Weight
Chronic Insomnia,Accident risk,20
Bullying,Alexithymia,30
Compensation,Alexithymia,75
Dysregulation,Alexithymia,50
Gaslighting,Alexithymia,40
Interoception Issues,Alexithymia,50
Shunning,Alexithymia,20
Bipolar Disorder,Anxiety,10
Bullying,Anxiety,30
Chronic Insomnia,Anxiety,20
Dysregulation,Anxiety,50
Gaslighting,Anxiety,40
Psychosis,Anxiety,10
Schizophrenia,Anxiety,10
Shunning,Anxiety,20
Toxic Stress,Asthma,10
Toxic Stress,Autoimmune Diseases,10
Bullying,Avoidance,30
Gaslighting,Avoidance,40
Shunning,Avoidance,20
Childhood Toxic Stress,Bipolar Disorder,30
Psychosis,Bipolar Disorder,10
Toxic Stress,Bipolar Disorder,10
Allergies,Brain Fog,50
Autoimmune Diseases,Brain Fog,50
Chronic Insomnia,Brain Fog,100
Cold,Brain Fog,50
Toxic Stress,Brain Fog,10
Virus,Brain Fog,50
Trouble Maintaining Relationships,Bullying,10
Allergies,Burn(out),25
Assimilation,Burn(out),100
Bullying,Burn(out),100
Cold,Burn(out),25
Compensation,Burn(out),40
Depression,Burn(out),30
Gaslighting,Burn(out),100
Headache/Migraine,Burn(out),25
Interoception Issues,Burn(out),50
Masking,Burn(out),30
Meltdown,Burn(out),25
Shunning,Burn(out),100
Shutdown,Burn(out),50
Virus,Burn(out),25
Bipolar Disorder,Cancer,10
Chronic Infection,Cancer,1
Susceptible to viral infections,Cancer,10
Psychosis,Cancer,10
Schizophrenia,Cancer,10
Toxic Stress,Chronic Infection,10
Depression,Chronic Insomnia,30
Interoception Issues,Chronic Insomnia,50
Toxic Stress,Chronic Insomnia,10
Allergies,Depression,50
Autoimmune Diseases,Depression,10
Bipolar Disorder,Depression,100
Bullying,Depression,75
Childhood Toxic Stress,Depression,100
Chronic Infection,Depression,75
Chronic Insomnia,Depression,75
Cold,Depression,50
Dysregulation,Depression,75
Fibromyalgia,Depression,100
Gaslighting,Depression,50
Headache/Migraine,Depression,75
Isolation,Depression,100
Psychosis,Depression,10
Schizophrenia,Depression,75
Shunning,Depression,75
Toxic Stress,Depression,100
Trouble Maintaining Relationships,Depression,100
Virus,Depression,50
Allergies,Difficulty Concentrating,25
Autoimmune Diseases,Difficulty Concentrating,50
Chronic Insomnia,Difficulty Concentrating,100
Cold,Difficulty Concentrating,25
Headache/Migraine,Difficulty Concentrating,100
Toxic Stress,Difficulty Concentrating,50
Virus,Difficulty Concentrating,25
Autoimmune Diseases,Digestive Problems,10
Chronic Insomnia,Digestive Problems,20
Toxic Stress,Digestive Problems,10
Autoimmune Diseases,Dry skin/sensitivity to cold,10
Allergies,Dysregulation,50
Bipolar Disorder,Dysregulation,100
Bullying,Dysregulation,75
Childhood Toxic Stress,Dysregulation,100
Chronic Infection,Dysregulation,100
Chronic Insomnia,Dysregulation,100
Cold,Dysregulation,50
Depression,Dysregulation,100
Fibromyalgia,Dysregulation,50
Gaslighting,Dysregulation,75
Headache/Migraine,Dysregulation,50
Interoception Issues,Dysregulation,50
Isolation,Dysregulation,100
Psychosis,Dysregulation,10
Schizophrenia,Dysregulation,100
Shunning,Dysregulation,75
Toxic Stress,Dysregulation,75
Trouble Maintaining Relationships,Dysregulation,100
Virus,Dysregulation,50
Dysregulation,Executive Function,50
Allergies,Fatigue,50
Autoimmune Diseases,Fatigue,50
Chronic Insomnia,Fatigue,100
Cold,Fatigue,50
Fibromyalgia,Fatigue,100
Headache/Migraine,Fatigue,100
Interoception Issues,Fatigue,50
Toxic Stress,Fatigue,50
Virus,Fatigue,50
Autoimmune Diseases,Fibromyalgia,10
Depression,Fibromyalgia,30
Interoception Issues,Fibromyalgia,50
Trouble Maintaining Relationships,Gaslighting,10
Allergies,Headache/Migraine,50
Chronic Insomnia,Headache/Migraine,50
Cold,Headache/Migraine,50
Depression,Headache/Migraine,50
Fibromyalgia,Headache/Migraine,50
Toxic Stress,Headache/Migraine,50
Virus,Headache/Migraine,100
Bipolar Disorder,Heart Disease,10
Chronic Insomnia,Heart Disease,20
Psychosis,Heart Disease,10
Schizophrenia,Heart Disease,10
Chronic Insomnia,High Blood Pressure,20
Bipolar Disorder,Homelessness,10
Bullying,Homelessness,30
Gaslighting,Homelessness,40
Psychosis,Homelessness,100
Schizophrenia,Homelessness,10
Shunning,Homelessness,20
Substance Abuse,Homelessness,50
Toxic Stress,Increased risk of diabetes,10
Toxic Stress,increased risk of ulcerative colitis,10
Toxic Stress,Increased time to recover from illness,10
Alexithymia,Interoception Issues,5
Bullying,Interoception Issues,30
Compensation,Interoception Issues,75
Depression,Interoception Issues,30
Dysregulation,Interoception Issues,50
Gaslighting,Interoception Issues,40
Shunning,Interoception Issues,20
Allergies,Irritability,50
Chronic Insomnia,Irritability,100
Cold,Irritability,25
Depression,Irritability,75
Dysregulation,Irritability,75
Headache/Migraine,Irritability,100
Interoception Issues,Irritability,50
Toxic Stress,Irritability,50
Virus,Irritability,25
Allergies,Isolation,75
Cold,Isolation,75
Depression,Isolation,30
Dysregulation,Isolation,50
Fibromyalgia,Isolation,50
Headache/Migraine,Isolation,100
Stigma,Isolation,100
Trouble Maintaining Relationships,Isolation,10
Virus,Isolation,75
Allergies,Loneliness,75
Avoidance,Loneliness,100
Bullying,Loneliness,50
Cold,Loneliness,75
Gaslighting,Loneliness,50
Headache/Migraine,Loneliness,100
Shunning,Loneliness,100
Virus,Loneliness,75
Allergies,Meltdown,50
Assimilation,Meltdown,100
Bullying,Meltdown,100
Chronic Insomnia,Meltdown,100
Cold,Meltdown,50
Compensation,Meltdown,75
Depression,Meltdown,50
Dysregulation,Meltdown,50
Gaslighting,Meltdown,100
Headache/Migraine,Meltdown,100
Interoception Issues,Meltdown,50
Masking,Meltdown,50
Shunning,Meltdown,100
Shutdown,Meltdown,25
Virus,Meltdown,50
Toxic Stress,Muscle Weakness,10
Chronic Insomnia,Poor School/Job Performance,20
Childhood Toxic Stress,Psychosis,30
Toxic Stress,Psychosis,10
Childhood Toxic Stress,PTSD,30
Toxic Stress,PTSD,10
Autoimmune Diseases,Recurring fever,10
Childhood Toxic Stress,Schizophrenia,30
Psychosis,Schizophrenia,10
Toxic Stress,Schizophrenia,10
Trouble Maintaining Relationships,Shunning,10
Allergies,Shutdown,50
Assimilation,Shutdown,100
Bullying,Shutdown,100
Cold,Shutdown,50
Compensation,Shutdown,75
Depression,Shutdown,50
Gaslighting,Shutdown,100
Headache/Migraine,Shutdown,100
Interoception Issues,Shutdown,50
Masking,Shutdown,30
Meltdown,Shutdown,75
Shunning,Shutdown,100
Virus,Shutdown,50
Bipolar Disorder,SI->MD,10
Bullying,SI->MD,30
Depression,SI->MD,30
Dysregulation,SI->MD,50
Gaslighting,SI->MD,40
Psychosis,SI->MD,10
Schizophrenia,SI->MD,10
Shunning,SI->MD,20
Allergies,Skin Problems,100
Autoimmune Diseases,Skin Problems,100
Cold,Skin Problems,50
Toxic Stress,Skin Problems,75
Virus,Skin Problems,50
Anxiety,Social Anxiety,5
Bipolar Disorder,Social Anxiety,10
Bullying,Social Anxiety,30
Chronic Insomnia,Social Anxiety,20
Gaslighting,Social Anxiety,40
Psychosis,Social Anxiety,100
Schizophrenia,Social Anxiety,10
Shunning,Social Anxiety,20
Bullying,Stigma,30
Gaslighting,Stigma,40
Isolation,Stigma,10
Shunning,Stigma,20
Trouble Maintaining Relationships,Stigma,10
Bipolar Disorder,Substance Abuse,10
Bullying,Substance Abuse,30
Chronic Insomnia,Substance Abuse,20
Dysregulation,Substance Abuse,50
Gaslighting,Substance Abuse,40
Psychosis,Substance Abuse,75
Schizophrenia,Substance Abuse,10
Shunning,Substance Abuse,20
Toxic Stress,Susceptible to viral infections,10
Autoimmune Diseases,Swollen glands,100
Cold,Swollen glands,100
Virus,Swollen glands,100
Bullying,Toxic Stress,30
Childhood Toxic Stress,Toxic Stress,30
Gaslighting,Toxic Stress,40
Shunning,Toxic Stress,20
Stigma,Toxic Stress,100
Bipolar Disorder,Trouble Maintaining Relationships,30
Bullying,Trouble Maintaining Relationships,30
Childhood Toxic Stress,Trouble Maintaining Relationships,30
Depression,Trouble Maintaining Relationships,30
Dysregulation,Trouble Maintaining Relationships,30
Gaslighting,Trouble Maintaining Relationships,30
Psychosis,Trouble Maintaining Relationships,75
Schizophrenia,Trouble Maintaining Relationships,30
Shunning,Trouble Maintaining Relationships,30
Toxic Stress,Trouble Maintaining Relationships,30
Autoimmune Diseases,Weight changes,10
Bipolar Disorder,Weight changes,10
Psychosis,Weight changes,10
Schizophrenia,Weight changes,10
Toxic Stress,Weight changes,10
Allergies,Working Memory Issues,75
Autoimmune Diseases,Working Memory Issues,10
Chronic Insomnia,Working Memory Issues,20
Cold,Working Memory Issues,75
Headache/Migraine,Working Memory Issues,100
Toxic Stress,Working Memory Issues,100
Virus,Working Memory Issues,75
"""
csvStringIO = StringIO(csvString)
data = pd.read_csv(csvStringIO, sep=",")


In [74]:
graph_name = "autistic_toxic_stress_model"
graph_color = "#97c2fc"

net = Network(height="1000px", width="1000px", bgcolor="#222222", font_color="white", notebook=True, select_menu=True, filter_menu=True, cdn_resources='remote')
#net = net.Network(height="750px", width="100%", notebook=True)

# set the physics layout of the network
#data = pd.read_csv("shunning-isolation.csv")[["Source", "Target", "Weight"]]

sources = data['Source']
targets = data['Target']
weights = data['Weight']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]
    
    graph_color = "#97c2fc"
    if dst == "SI->MD" or dst == "Homelessness" or dst == "Burn(out)":
        graph_color = "red"
    net.add_node(src, src, title=src, size=w)
    net.add_node(dst, dst, title=dst, color=graph_color)
    net.add_edge(src, dst, value=w, color=graph_color)

neighbor_map = net.get_adj_list()

# add neighbor data to node hover data
for node in net.nodes:
    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

net.save_graph(f"/kevin/code/jupyter/{graph_name}.html")

# Read the contents of the HTML file
#net.show_buttons(filter_=["physics"])
net.show(f"{graph_name}.html")

autistic_toxic_stress_model.html
